In [1]:
!pip install torch torchaudio transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import soundfile as sf
import torchaudio

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [88]:
# Load Common Voice dataset
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="validation+train+test")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [89]:
print(dataset)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 49021
})


In [90]:
# Removing unwanted features (we only want audio and its transcription)
dataset = dataset.remove_columns(["client_id", "path", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 49021
})


In [91]:
print(dataset[0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d68675862a8f904b8c115f046d22a9eb133d1b8d146881ef631041738b0d8f3e/ar_dev_0/common_voice_ar_24079904.mp3', 'array': array([-3.69482223e-13,  6.82121026e-13,  1.42108547e-12, ...,
       -8.46983676e-06, -1.07555734e-05, -5.13326540e-06]), 'sampling_rate': 48000}, 'sentence': 'وَقْفَةُ الأهرامِ فيما بَيْنَنَا لِصُروفِ الدَّهرِ وَقْفَتى أنا'}


In [94]:
vocabulary = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ء', 'آ', 'أ', 'إ', 'ؤ', 'ئ', 'ة', 'ى', 'ﻻ', 'ﻷ', 'ﻹ', 'ﻵ',' ', '.']

In [95]:
vocabulary_set = set(vocabulary)  # Convert your existing vocabulary list to a set

# Initialize an empty set for characters from the dataset
dataset_characters = set()

for data_item in dataset:
    sentence = data_item['sentence']
    dataset_characters.update(sentence)

# Perform a union of the dataset characters with the existing vocabulary set
combined_vocabulary = vocabulary_set.union(dataset_characters)

# If you need the combined vocabulary as a list
combined_vocabulary_list = list(combined_vocabulary)

In [96]:
print(len(combined_vocabulary_list))

126


In [98]:
print(combined_vocabulary_list)

['—', 'ؤ', 'ی', 'ة', ':', 'X', 's', 'h', "'", 'G', '،', '☭', 'T', 'ۛ', 'ۗ', 'آ', '”', ')', 'ب', '"', 'B', '؟', '(', 'A', 'ظ', 'f', 'ج', 'a', 'ڨ', 'C', 't', 'م', 'ك', 'ى', 'ـ', ',', 'ه', 'c', 'ل', 'ھ', 'n', 'k', 'ط', '_', 'ّ', 'ﻷ', 'ذ', 'ِ', 'ف', 'ﻹ', 'ﻵ', 'ْ', 'm', 'x', 'u', '؛', 'p', 'ع', 'ز', 'e', 'd', 'r', 'ن', 'D', 'چ', ';', 'I', 'w', 'َ', 'س', 'غ', 'Q', 'ر', '?', 'ۘ', 'ۚ', 'M', 'S', 'ٍ', 'أ', 'y', '.', 'R', '!', 'و', 'ء', 'H', 'د', 'P', 'ۖ', 'ً', '«', 'E', '-', 'ش', 'ت', 'إ', 'l', 'v', 'U', 'V', ' ', 'ض', '“', 'ص', 'ٌ', 'b', 'ک', 'ٰ', '…', 'j', 'ئ', 'خ', 'i', 'o', 'K', 'g', 'ح', 'ي', '»', 'ﻻ', 'ﺃ', 'ا', 'ُ', 'ث', 'ق']


In [99]:
combined_vocabulary_list.append('special')

In [100]:
print(combined_vocabulary_list[-1])

special


In [108]:
class CustomProcessor:
    def __init__(self, vocab):
        self.vocab = vocab
        self.char_to_index = {char: index for index, char in enumerate(vocab)}

    def text_to_int(self, text):
        return [self.char_to_index[char] for char in text]

    def int_to_text(self, indices):
        return ''.join([self.vocab[index] for index in indices])

In [109]:
object_voc = CustomProcessor(combined_vocabulary_list)

In [110]:
text = "مرحبا"
encoding = object_voc.text_to_int(text)
decoding = object_voc.int_to_text(encoding)
print(encoding)
print(decoding)

[31, 72, 117, 18, 122]
مرحبا


In [107]:
import torch
from torch.utils.data import Dataset
import torchaudio

In [178]:
class CommonVoiceDataset(Dataset):
    def __init__(self, dataset, vocab):
        self.dataset = dataset
        self.processor = CustomProcessor(vocab)
        self.spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load and resample audio
        audio_input, sampling_rate = torchaudio.load(self.dataset[idx]["audio"]["path"])
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio_input = resampler(audio_input)

        # Convert to spectrogram
        # spectrogram = self.spectrogram_transform(audio_input).squeeze(0)
        spectrogram = self.spectrogram_transform(audio_input)
        # Normalize spectrogram
        spectrogram = (spectrogram - spectrogram.mean()) / spectrogram.std()
        print(spectrogram.shape)

        # Process text
        sentence = self.dataset[idx]["sentence"]

        labels = self.processor.text_to_int(sentence)
        labels = torch.tensor(labels)
        print(labels.shape)

        return spectrogram, labels, spectrogram.shape[-1]

In [179]:
# Instantiate the dataset
common_voice_dataset = CommonVoiceDataset(dataset, combined_vocabulary_list)

In [180]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [181]:
import torch
import torch.nn as nn

class CustomSTTModel2(nn.Module):
    def __init__(self, num_classes):
        super(CustomSTTModel2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.gru1 = nn.GRU(input_size=64, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(input_size=256, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.5)

        self.fc = nn.Linear(256, 256)
        self.out = nn.Linear(256, num_classes)  # num_classes includes the Arabic charset + 1 for CTC blank

    def forward(self, x):
        print("cccccccccccccccccccccccccc")
        print(x.shape)
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        print("dddddddddddddddddddddddddddd")
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))

        x = x.permute(0, 3, 1, 2)  # Rearrange dimensions for RNN
        x = torch.flatten(x, start_dim=2)  # Flatten the convolutional features

        x, _ = self.gru1(x)
        x = self.dropout(x)
        x, _ = self.gru2(x)
        x = self.dropout(x)

        x = torch.relu(self.fc(x))
        x = self.dropout(x)
        x = self.out(x)
        return x

In [182]:
# Instantiate the model
custom_model_2 = CustomSTTModel2(len(combined_vocabulary_list))

In [183]:
# Configuration
batch_size = 32
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [184]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    # Sort the batch in the descending order of sequence length
    batch = sorted(batch, key=lambda x: x[2], reverse=True)

    # Separate the data items
    spectrograms, labels, input_lengths = zip(*batch)
    print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
    # Pad the spectrograms to have the same length
    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True)
    print("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")

    # Concatenate all labels (no need for padding)
    labels = torch.cat(labels)

    # Convert input_lengths into a tensor
    input_lengths = torch.tensor(input_lengths)

    return spectrograms, labels, input_lengths

In [190]:
# Create a DataLoader
batch_size = 8 # Adjust as needed
train_loader = DataLoader(common_voice_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [191]:
def train(model, data_loader, criterion, optimizer, epochs, device):
  model.to(device)
  for epoch in range(num_epochs):
      model.train()  # Set the model to training mode
      total_loss = 0

      for batch_idx, (spectrograms, labels, input_lengths) in enumerate(train_loader):
          spectrograms, labels = spectrograms.to(device), labels.to(device)

          # Forward pass
          outputs = model(spectrograms)

          # The output of the network needs to be in the shape (T, N, C)
          outputs = outputs.permute(2, 0, 1)  # Change to (time, batch, class)
          output_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.size(0), dtype=torch.long)

          # Calculate loss
          loss = criterion(outputs, labels, output_lengths, input_lengths)
          total_loss += loss.item()

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          # Print progress
          if (batch_idx + 1) % 100 == 0:
              print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

      # Average loss for this epoch
      avg_loss = total_loss / len(train_loader)
      print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

In [192]:
from torch import nn
import torch.optim as optim

# Define the Loss Function (Criterion)
# The CTC loss function expects logits as inputs, so ensure your model outputs logits
criterion = nn.CTCLoss(blank=combined_vocabulary_list[-1]).to(device)

In [193]:
# Define the Optimizer
optimizer = optim.Adam(custom_model_2.parameters(), lr=0.001)

In [194]:
# Run the training
epochs = 10  # Define the number of epochs
train(custom_model_2, train_loader, criterion, optimizer, epochs, device)

torch.Size([1, 128, 479])
torch.Size([64])
torch.Size([1, 128, 260])
torch.Size([16])
torch.Size([1, 128, 464])
torch.Size([46])
torch.Size([1, 128, 323])
torch.Size([29])
torch.Size([1, 128, 283])
torch.Size([27])
torch.Size([1, 128, 274])
torch.Size([30])
torch.Size([1, 128, 240])
torch.Size([24])
torch.Size([1, 128, 355])
torch.Size([55])
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


RuntimeError: ignored